### Import all libraries

In [3]:
from __future__ import print_function
import os
import collections
import numpy as np
import six
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math
import cv2
import matplotlib.pyplot as plt
import SimpleITK as sitk
import scipy.io as sio
import openpyxl
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import cnames
from itertools import cycle
from bokeh.plotting import output_notebook, output_file, figure, show, ColumnDataSource
from bokeh.models import HoverTool
import warnings
from bokeh.palettes import PuBu
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, ranges, LabelSet
from bokeh.plotting import figure
from sklearn import metrics
warnings.filterwarnings(action='ignore')
output_notebook()
%matplotlib inline

Loading BokehJS ...

### PCA and accuracy

In [6]:
# Here fractures = 1 and pores = 0
num_frac = [1, 2, 3]

df_frac = pd.DataFrame()
df_true = pd.DataFrame()

for num in num_frac:
    csv_name = 'D:\\Fracture Identification\\version_3\\Properties_Excel\\kmeans_ben_acc{}.csv'.format(num)
    df = pd.read_csv(csv_name)
    df_frac = df_frac.append(df)
    csv_name_true = 'D:\\Fracture Identification\\version_3\\Properties_Excel\\True Label\\true_ben{}.csv'.format(num)
    df_t = pd.read_csv(csv_name_true)
    df_true = df_true.append(df_t)

In [8]:
df_frac['Fracture_PCA2'].value_counts()

1.0    152077
0.0       896
Name: Fracture_PCA2, dtype: int64

In [9]:
df_true['True_Label'].value_counts()

1    152073
0       900
Name: True_Label, dtype: int64

In [10]:
adj_rand_idx = []
adj_mi = []
homo = []
comp = []
v_measure = []


for pca_num in range(2, 15):
    
    col_name_frac = 'Fracture_PCA{}'.format(pca_num)
    
    adj_rand_idx.append(metrics.adjusted_rand_score(df_true['True_Label'], df_frac[col_name_frac]))
    adj_mi.append(metrics.adjusted_mutual_info_score(df_true['True_Label'], df_frac[col_name_frac]))
    homo.append(metrics.homogeneity_score(df_true['True_Label'], df_frac[col_name_frac]))
    comp.append(metrics.completeness_score(df_true['True_Label'], df_frac[col_name_frac]))
    v_measure.append(metrics.v_measure_score(df_true['True_Label'], df_frac[col_name_frac], beta=0.5))
    
    
print('Adjusted Rand Index:', adj_rand_idx)
print('Adjusted Mutual Information:', adj_mi)
print('Homogeneity Score:', homo)
print('Completeness Score:', comp)
print('V measure score:', v_measure)

Adjusted Rand Index: [0.9977332790593567, 0.9977332790593567, 0.9977332790593567, 0.9977332790593567, 0.9977332790593567, 0.9977332790593567, 0.9977332790593567, 0.9983008775163158, 0.9977332790593567, 0.9977332790593567, 0.9977332790593567, 0.9977332790593567, 0.9977332790593567]
Adjusted Mutual Information: [0.9916317635099762, 0.9916317635099762, 0.9916317635099762, 0.9916317635099762, 0.9916317635099762, 0.9916317635099762, 0.9916317635099762, 0.9935674439337194, 0.9916317635099762, 0.9916317635099762, 0.9916317635099762, 0.9916317635099762, 0.9916317635099762]
Homogeneity Score: [0.9916325542281726, 0.9916325542281726, 0.9916325542281726, 0.9916325542281726, 0.9916325542281726, 0.9916325542281726, 0.9916325542281726, 0.9935680517084263, 0.9916325542281726, 0.9916325542281726, 0.9916325542281726, 0.9916325542281726, 0.9916325542281726]
Completeness Score: [0.9953344129469984, 0.9953344129469984, 0.9953344129469984, 0.9953344129469984, 0.9953344129469984, 0.9953344129469984, 0.99533

In [11]:
adj_rand_idx = [ 100 * elem for elem in adj_rand_idx]
adj_rand_idx= [ '%.2f' % elem for elem in adj_rand_idx]

homo = [ 100 * elem for elem in homo]
homo= [ '%.2f' % elem for elem in homo]

In [12]:
pca_columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
from bokeh.plotting import figure

p = figure()

p = figure(x_range=pca_columns,  plot_width = 800, plot_height=400, title="Metrics for different PCs: Fractured Bentheimer Sandstone",
           toolbar_location=None, tools="", y_range= ranges.Range1d(start=95,end=100))

p.line(x=pca_columns, y=adj_rand_idx, color="firebrick", legend_label = "Adjusted Rand Index (ARI)")
p.line(x=pca_columns, y=homo, color="navy", legend_label = "Homogeneity", line_dash=[6, 6])

show(p)